In [1]:
import pandas as pd
import numpy as np

In [2]:
datatypes = {'Code département': str, 'Code du département': str, 'circonscription':str, 'Code de la circonscription':str}

In [3]:
data1965 = pd.read_csv("..\cdsp_presi1965t1_circ.csv", dtype = datatypes)
data1969 = pd.read_csv("..\cdsp_presi1969t1_circ.csv", dtype = datatypes)
data1974 = pd.read_csv("..\cdsp_presi1974t1_circ.csv", dtype = datatypes)
data1981 = pd.read_csv("..\cdsp_presi1981t1_circ.csv", dtype = datatypes)
data1988 = pd.read_csv("..\cdsp_presi1988t1_circ.csv", dtype = datatypes)
data1995 = pd.read_csv("..\cdsp_presi1995t1_circ.csv", dtype = datatypes)
data2002 = pd.read_csv("..\cdsp_presi2002t1_circ.csv", dtype = datatypes)
data2007 = pd.read_csv("..\cdsp_presi2007t1_circ.csv", dtype = datatypes)
data2012 = pd.read_csv("..\cdsp_presi2012t1_circ.csv", dtype = datatypes)
data2012 = data2012.rename (columns = {'Code du département' : 'Code département', 'Libellé du département' : 'département','Code de la circonscription': 'circonscription'})

In [4]:
listedatasets = [data1965, data1969, data1974, data1981, data1988, data1995, data2002, data2007, data2012]
year = ['1965','1969', '1974', '1981', '1988', '1995', '2002', '2007', '2012']

# concaténation de tous les fichiers, après les avoir "aplatis"
# => constitué une colonne avec tous les candidats plutôt que de multiplier les colonnes (cf "melt"))

all_datasets = [] 
for dataset,layear in zip(listedatasets, year):
    dataset.drop('Code département', axis=1, inplace=True)
    dataset['Année'] = layear
    dataset = dataset.melt( id_vars=['département', 'circonscription', 'Année', 'Inscrits','Votants','Exprimés',
       'Blancs et nuls'], var_name = 'Candidat', value_name = 'Nb_votants')
    all_datasets.append(dataset)

data = pd.concat(all_datasets, axis=0, sort=False)
data.head()

,département,circonscription,Année,Inscrits,Votants,Exprimés,Blancs et nuls,Candidat,Nb_votants
0,AIN,1,1965,68818,54262,53769,493,MITTERRAND (CIR),16677
1,AIN,2,1965,70186,57221,56780,441,MITTERRAND (CIR),18247
2,AIN,3,1965,67492,55503,55006,497,MITTERRAND (CIR),15494
3,AISNE,1,1965,52618,46064,45564,500,MITTERRAND (CIR),12999
4,AISNE,2,1965,68770,59908,59145,763,MITTERRAND (CIR),19612


In [5]:
# nettoyage du champ candidat, pour en récupérer 2 colonnes : Une avec le nom du candidat, une avec son parti

data['Parti'] = data.Candidat.apply(lambda x: x.split(" (")[1].replace(")","") if "(" in x else np.nan)
data.Candidat = data.Candidat.apply(lambda x: x.split(" (")[0].replace(")","") if "(" in x else x)

In [6]:
# Le fichier de 2012 est configuré un peu différemment => transformation des noms pour les uniformiser avec ceux des autres fichiers
# (sauf Marine Le Pen, pour la distinguer de son père)

data.Candidat = data.Candidat.replace('Eva JOLY','JOLY').replace('François BAYROU','BAYROU').replace('Jacques CHEMINADE','CHEMINADE').replace('François HOLLANDE','HOLLANDE').replace('Jean-Luc MELENCHON','MELENCHON').replace('François HOLLANDE','HOLLANDE').replace('Nathalie ARTHAUD','ARTHAUD').replace('Nicolas DUPONT-AIGNAN','DUPONT-AIGNAN').replace('Nicolas SARKOZY','SARKOZY').replace('Philippe POUTOU','POUTOU')

In [7]:
data['Année'] = data['Année'].apply(int)

In [8]:
# constitution d'une nouvelle variable, qui regarde si le candidat s'est déjà présenté, et combien de fois.

ranklookup = pd.concat([data[['Année', 'Candidat']].drop_duplicates(), data[['Année', 'Candidat']].drop_duplicates().groupby('Candidat').rank()], axis = 1)
ranklookup.columns = ['Année', 'Candidat', 'Rank']
datanew = data.merge(ranklookup, on = ['Année', 'Candidat'], how = 'left')

In [11]:
datanew.head()

,département,circonscription,Année,Inscrits,Votants,Exprimés,Blancs et nuls,Candidat,Nb_votants,Parti,Rank
0,AIN,1,1965,68818,54262,53769,493,MITTERRAND,16677,CIR,1.0
1,AIN,2,1965,70186,57221,56780,441,MITTERRAND,18247,CIR,1.0
2,AIN,3,1965,67492,55503,55006,497,MITTERRAND,15494,CIR,1.0
3,AISNE,1,1965,52618,46064,45564,500,MITTERRAND,12999,CIR,1.0
4,AISNE,2,1965,68770,59908,59145,763,MITTERRAND,19612,CIR,1.0


In [12]:
datanew.to_excel("Elections_5e_Rep_1ers_Tours.xlsx")